# Gradnja z modeli iz družine Meta

## Uvod

V tej lekciji bomo obravnavali:

- Raziskovanje dveh glavnih modelov iz družine Meta – Llama 3.1 in Llama 3.2
- Razumevanje primerov uporabe in scenarijev za vsak model
- Primer kode, ki prikazuje edinstvene lastnosti vsakega modela

## Družina modelov Meta

V tej lekciji bomo raziskali 2 modela iz družine Meta oziroma "Llama Herd" – Llama 3.1 in Llama 3.2

Ti modeli so na voljo v različnih različicah in jih najdete na tržnici Github Model. Več informacij o uporabi Github Models za [prototipiranje z AI modeli](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Različice modelov:
- Llama 3.1 – 70B Instruct
- Llama 3.1 – 405B Instruct
- Llama 3.2 – 11B Vision Instruct
- Llama 3.2 – 90B Vision Instruct

*Opomba: Llama 3 je prav tako na voljo na Github Models, vendar v tej lekciji ne bo obravnavana*


## Llama 3.1

S 405 milijardami parametrov Llama 3.1 spada v kategorijo odprtokodnih LLM-jev.

Ta model je nadgradnja prejšnje različice Llama 3 in prinaša:

- Večje kontekstno okno – 128k žetonov v primerjavi z 8k žetoni
- Večje največje število izhodnih žetonov – 4096 v primerjavi z 2048
- Boljša podpora za več jezikov – zaradi večjega števila učnih žetonov

To omogoča, da Llama 3.1 obvladuje bolj zahtevne primere uporabe pri razvoju GenAI aplikacij, vključno z:
- Izvorno klicanje funkcij – možnost klicanja zunanjih orodij in funkcij izven LLM delovnega toka
- Boljša RAG zmogljivost – zaradi večjega kontekstnega okna
- Generiranje sintetičnih podatkov – možnost ustvarjanja učinkovitih podatkov za naloge, kot je fino prilagajanje


### Klicanje izvornih funkcij

Llama 3.1 je bila dodatno izurjena, da je bolj učinkovita pri klicanju funkcij ali orodij. Ima tudi dve vgrajeni orodji, ki jih model lahko prepozna kot potrebna glede na uporabniški poziv. Ta orodja sta:

- **Brave Search** – Uporablja se lahko za pridobivanje najnovejših informacij, kot je vreme, z iskanjem po spletu
- **Wolfram Alpha** – Uporablja se lahko za bolj zapletene matematične izračune, tako da ni treba pisati lastnih funkcij.

Lahko pa ustvarite tudi svoja orodja, ki jih lahko LLM pokliče.

V spodnjem primeru kode:

- V sistemskem pozivu določimo razpoložljiva orodja (brave_search, wolfram_alpha).
- Pošljemo uporabniški poziv, ki sprašuje po vremenu v določenem mestu.
- LLM bo odgovoril s klicem orodja Brave Search, ki bo izgledal takole: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Opomba: Ta primer izvede le klic orodja. Če želite pridobiti rezultate, si morate ustvariti brezplačen račun na strani Brave API in definirati samo funkcijo.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Kljub temu, da je LLM, ima Llama 3.1 eno omejitev – multimodalnost. To pomeni, da ne zna uporabljati različnih vrst vhodov, kot so slike, kot pozive in podajati odgovore. Ta sposobnost je ena glavnih novosti v Llama 3.2. Med novimi funkcijami so:

- Multimodalnost – omogoča ocenjevanje tako besedilnih kot slikovnih pozivov
- Različice od majhnih do srednje velikih (11B in 90B) – to omogoča prilagodljive možnosti uporabe,
- Samo besedilne različice (1B in 3B) – to omogoča uporabo modela na robnih / mobilnih napravah in zagotavlja nizko zakasnitev

Podpora za multimodalnost predstavlja velik korak v svetu odprtokodnih modelov. Spodnji primer kode sprejme tako sliko kot besedilni poziv, da pridobi analizo slike iz Llama 3.2 90B.

### Podpora za multimodalnost z Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Učenje se tukaj ne konča, nadaljujte svojo pot

Ko zaključite to lekcijo, si oglejte našo [zbirko za učenje generativne umetne inteligence](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) in še naprej nadgrajujte svoje znanje o generativni umetni inteligenci!



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko samodejni prevodi vsebujejo napake ali netočnosti. Izvirni dokument v svojem maternem jeziku naj velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
